<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime,timedelta

In [ ]:
customers_df = pd.DataFrame({
    'customer_id': [101, 102, 103, 104, 105, 106],
    'name': ['Amit', 'Bella', 'Chirag', 'Divya', 'Eshan', 'Farah'],
    'region': ['North', 'South', 'East', 'West', None, 'North'],
    'joined_date': pd.to_datetime(['2023-01-15', '2023-03-10', '2023-05-20', '2023-07-01', '2023-02-28', '2023-08-05']),
})

orders_df = pd.DataFrame({
    'order_id': [5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008, 5009],
    'customer_id': [101, 105, 103, 102, 101, 104, 106, 106, 101],
    'order_amount': [2500, 4000, 1200, 3800, 2750, 6600, 1800, 1900, 3100],
    'order_date': pd.to_datetime(['2023-04-10', '2023-06-15', '2023-08-05', '2023-09-12',
                                  '2023-10-21', '2023-11-01', '2023-08-06', '2023-09-10', '2023-03-01']),
    'product_category': ['Electronics', 'Clothing', 'Home', 'Clothing', 'Electronics',
                         'Home', 'Electronics', 'Electronics', 'Clothing']
})

returns_df = pd.DataFrame({
    'order_id': [5002, 5004, 5010, 5006],  # 5010 is an invalid ID for false positive detection
    'return_reason': ['Defective', 'Size Mismatch', 'No Record', 'Late Delivery'],
    'return_date': pd.to_datetime(['2023-06-10', '2023-08-10', '2023-04-01', '2023-10-20']),
})

In [ ]:
orders = orders_df.merge(returns_df,on='order_id',how='left')
orders['False_flags'] = orders.apply(lambda x:(True if x.order_date>x.return_date else False),axis=1)
orders[orders.False_flags==True]

,order_id,customer_id,order_amount,order_date,product_category,return_reason,return_date,False_flags
1,5002,105,4000,2023-06-15,Clothing,Defective,2023-06-10,True
3,5004,102,3800,2023-09-12,Clothing,Size Mismatch,2023-08-10,True
5,5006,104,6600,2023-11-01,Home,Late Delivery,2023-10-20,True


In [ ]:
category = orders_df['product_category'].nunique()
x = orders_df.groupby('customer_id')['product_category'].nunique().reset_index()
x[x.product_category==category]

,customer_id,product_category


In [ ]:
x = customers_df.merge(orders_df,on='customer_id',how='inner')
y = x.merge(returns_df,on='order_id',how='left')
y['orders_per_region'] = y.groupby('region')['order_id'].transform('count').fillna(0).astype(int)
y['return_per_region'] = y.groupby('region')['return_reason'].transform('count').fillna(0).astype(int)
y = y[y.return_per_region>0].loc[:,['region','orders_per_region','return_per_region']]
y['revenue_dropoff']  = (y['orders_per_region']/y['return_per_region'])*100
y

,region,orders_per_region,return_per_region,revenue_dropoff
3,South,1,1,100.0
5,West,1,1,100.0


In [ ]:
x[x.order_date<(x.joined_date+pd.Timedelta(days=31))]

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category
7,106,Farah,North,2023-08-05,5007,1800,2023-08-06,Electronics


In [ ]:
x['diverse_buys'] = x.groupby('customer_id')['product_category'].transform('nunique')
x['diverse_buys'] = x['diverse_buys'].map({1:'Loyal'})
x['diverse_buys'] = x['diverse_buys'].fillna('Diverse')
x

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category,diverse_buys
0,101,Amit,North,2023-01-15,5001,2500,2023-04-10,Electronics,Diverse
1,101,Amit,North,2023-01-15,5005,2750,2023-10-21,Electronics,Diverse
2,101,Amit,North,2023-01-15,5009,3100,2023-03-01,Clothing,Diverse
3,102,Bella,South,2023-03-10,5004,3800,2023-09-12,Clothing,Loyal
4,103,Chirag,East,2023-05-20,5003,1200,2023-08-05,Home,Loyal
5,104,Divya,West,2023-07-01,5006,6600,2023-11-01,Home,Loyal
6,105,Eshan,None,2023-02-28,5002,4000,2023-06-15,Clothing,Loyal
7,106,Farah,North,2023-08-05,5007,1800,2023-08-06,Electronics,Loyal
8,106,Farah,North,2023-08-05,5008,1900,2023-09-10,Electronics,Loyal


In [ ]:
orderids = list(orders_df['order_id'].drop_duplicates())
orderids
returns_df[returns_df.order_id.isin(orderids)==False]

,order_id,return_reason,return_date
2,5010,No Record,2023-04-01


In [ ]:
orcus = customers_df.merge(orders_df,on='customer_id',how='inner')
orcus[orcus.region.isna()&orcus.order_id.notna()]

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category
6,105,Eshan,None,2023-02-28,5002,4000,2023-06-15,Clothing


In [ ]:
x['order_within_month'] = x[x.order_date<(x.joined_date+pd.Timedelta(days=31))].groupby('customer_id')['order_id'].transform('count')
x['order_within_month'] = x['order_within_month'].fillna(0).astype(int)

In [ ]:
x

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category,diverse_buys,order_within_month
0,101,Amit,North,2023-01-15,5001,2500,2023-04-10,Electronics,Diverse,0
1,101,Amit,North,2023-01-15,5005,2750,2023-10-21,Electronics,Diverse,0
2,101,Amit,North,2023-01-15,5009,3100,2023-03-01,Clothing,Diverse,0
3,102,Bella,South,2023-03-10,5004,3800,2023-09-12,Clothing,Loyal,0
4,103,Chirag,East,2023-05-20,5003,1200,2023-08-05,Home,Loyal,0
5,104,Divya,West,2023-07-01,5006,6600,2023-11-01,Home,Loyal,0
6,105,Eshan,None,2023-02-28,5002,4000,2023-06-15,Clothing,Loyal,0
7,106,Farah,North,2023-08-05,5007,1800,2023-08-06,Electronics,Loyal,1
8,106,Farah,North,2023-08-05,5008,1900,2023-09-10,Electronics,Loyal,0


In [ ]:
a = customers_df.merge(orders_df,on='customer_id',how='inner')
b  = a.merge(returns_df,on='order_id',how='left')
b['orderNums'] = b.groupby('customer_id')['order_id'].transform('count').astype(int)
b[(b.orderNums==1)&b.return_date.isna()]

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category,return_reason,return_date,orderNums
4,103,Chirag,East,2023-05-20,5003,1200,2023-08-05,Home,NaN,NaT,1


In [ ]:
orcus

,customer_id,name,region,joined_date,order_id,order_amount,order_date,product_category
0,101,Amit,North,2023-01-15,5001,2500,2023-04-10,Electronics
1,101,Amit,North,2023-01-15,5005,2750,2023-10-21,Electronics
2,101,Amit,North,2023-01-15,5009,3100,2023-03-01,Clothing
3,102,Bella,South,2023-03-10,5004,3800,2023-09-12,Clothing
4,103,Chirag,East,2023-05-20,5003,1200,2023-08-05,Home
5,104,Divya,West,2023-07-01,5006,6600,2023-11-01,Home
6,105,Eshan,None,2023-02-28,5002,4000,2023-06-15,Clothing
7,106,Farah,North,2023-08-05,5007,1800,2023-08-06,Electronics
8,106,Farah,North,2023-08-05,5008,1900,2023-09-10,Electronics


In [ ]:
reor = returns_df.merge(orders_df,on='order_id',how='inner')
reor['ret_per_cus'] = reor.groupby('customer_id')['return_reason'].transform('count').astype(int)
reor['ret_product'] = reor.groupby('customer_id')['product_category'].transform('nunique')
reor[(reor.ret_per_cus>1)&(reor.ret_product==1)]

,order_id,return_reason,return_date,customer_id,order_amount,order_date,product_category,ret_per_cus,ret_product


In [ ]:
reor = reor.iloc[:,:-3]

In [ ]:
reor

,order_id,return_reason,return_date,customer_id,order_amount,order_date
0,5002,Defective,2023-06-10,105,4000,2023-06-15
1,5004,Size Mismatch,2023-08-10,102,3800,2023-09-12
2,5006,Late Delivery,2023-10-20,104,6600,2023-11-01


In [ ]:
reor['avg_delay']  = pd.to_datetime(reor.order_date)-pd.to_datetime(reor.return_date)

In [ ]:
reor

,order_id,return_reason,return_date,customer_id,order_amount,order_date,avg_delay
0,5002,Defective,2023-06-10,105,4000,2023-06-15,5 days
1,5004,Size Mismatch,2023-08-10,102,3800,2023-09-12,33 days
2,5006,Late Delivery,2023-10-20,104,6600,2023-11-01,12 days


In [ ]:
# Introduce simulated duplicate for test case:
orders_df.loc[9] = orders_df.iloc[0]  # duplicate row

# Find duplicate order_ids
dupes = orders_df[orders_df.duplicated('order_id', keep=False)]

# Identify which fields differ among dupes
conflicts = dupes.groupby('order_id').nunique().reset_index()
conflicts = conflicts[conflicts.gt(1).any(axis=1)]

In [ ]:
conflicts

,order_id,customer_id,order_amount,order_date,product_category
0,5001,1,1,1,1


In [ ]:
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

# Sort by customer and date
sorted_orders = orders_df.sort_values(by=['customer_id', 'order_date'])

# First and last category per customer
category_switch = sorted_orders.groupby('customer_id')['product_category'].agg(['first', 'last']).reset_index()
category_switch['category_switched'] = category_switch['first'] != category_switch['last']

In [ ]:
category_switch

,customer_id,first,last,category_switched
0,101,Clothing,Electronics,True
1,102,Clothing,Clothing,False
2,103,Home,Home,False
3,104,Home,Home,False
4,105,Clothing,Clothing,False
5,106,Electronics,Electronics,False
